In [ ]:
import astropy.units as u
import lsdb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dask
import scipy

from dask.distributed import Client
from nested_pandas.utils import count_nested
from dask.distributed import print as dask_print
from lsdb.core.search.pixel_search import PixelSearch
from lsdb.core.search.order_search import OrderSearch
from io import StringIO
from nested_pandas import NestedDtype
from pathlib import Path

In [ ]:
drp_release = "w_2025_11"
hats_dir = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats")
hats_path = hats_dir / drp_release
output_path = Path(
    "/sdf/data/rubin/shared/lsdb_commissioning/science_projects/06_uncertainty/object_whiten"
)

In [ ]:
client = Client(n_workers=1, threads_per_worker=1, memory_limit="10GB")

dask.config.set({"dataframe.convert-string": False})

There's not that much going on here! We did all the work on all the objects and lightcurves in the previous notebook, so this is just aggregating the histograms over all lightcurves.

In [ ]:
x = np.arange(-10, 10, 0.01)
def fetch_and_plot_dev(band, plot=False, field="whitened_data"):
    stats_catalog = lsdb.read_hats(    output_path / f"{band}_band")

    def dev_histogram(frame):
        return {"hist": np.histogram(np.concatenate(frame[field].values).ravel(), bins=x)}

    result = stats_catalog.map_partitions(dev_histogram, meta={"hist":np.int64})
    full_hist = result.compute().loc[0].to_numpy().sum(axis=0)[0]

    if plot:
        _ = plt.hist(x[:-1], x, weights=full_hist, density=True)
        _ = plt.plot(x, scipy.stats.norm().pdf(x))

    return full_hist

In [ ]:
bands = ["u", "g", "r", "i", "z", "y"]

rel_dev_hists_100 = [fetch_and_plot_dev(band, field="rel_dev") for band in bands]

In [ ]:
whitened_hists_100 = [fetch_and_plot_dev(band, field="whitened_data") for band in bands]

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(20, 10), layout="constrained")
for ax, band, histo in zip(axs.flat, bands, rel_dev_hists):
    _ = ax.hist(x[:-1], x, weights=histo, density=True)
    _ = ax.plot(x, scipy.stats.norm().pdf(x))
    ax.set_title(band + " band", fontsize=20)
_=fig.suptitle('Relative Deviations in the full sky object forced photometry, by band', fontsize=20)

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(20, 10), layout="constrained")
for ax, band, histo in zip(axs.flat, bands, whitened_hists):
    _ = ax.hist(x[:-1], x, weights=histo, density=True)
    _ = ax.plot(x, scipy.stats.norm().pdf(x))
    ax.set_title(band + " band", fontsize=20)
_=fig.suptitle('Deviations in whitened in the full sky object forced photometry, by band', fontsize=20)

In [ ]:
fig, axs = plt.subplots(6, 2, figsize=(10, 30), layout="constrained")

for i in range(0, 6):
    ax = axs[i, 0]
    _ = ax.hist(x[:-1], x, weights=rel_dev_hists[i], density=True)
    _ = ax.plot(x, scipy.stats.norm().pdf(x))
    ax.set_title(bands[i] + " band (rel dev)", fontsize=20)

    ax = axs[i, 1]
    _ = ax.hist(x[:-1], x, weights=whitened_hists[i], density=True)
    _ = ax.plot(x, scipy.stats.norm().pdf(x))
    ax.set_title(bands[i] + " band (whitened)", fontsize=20)

_=fig.suptitle('Comparison of relative deviation and whitening in the full sky object forced photometry, by band', fontsize=20)

In [ ]:
fig.savefig("full_sky_method_compare.png")